## Fetch Weather Data: From Stations Around the Wind Farms
### *This notebook downloads wind data from the NOAA API from the list of stations found in 02 - Fetch Weather Data.ipynb*

In [9]:
import requests
import json
import os
import pandas as pd
import numpy as np
from pyproj import Proj
from shapely.geometry import shape
import pickle
headers = {"token": "xVEIkLnfHyheHhvoheZSxesUerlyrxGN"}
#headers = {"token": "bgBKBpxaFICMeDMgPFYAMGyaDrBJMPwB"}

#### Stations
* Stations are where the data comes from (for most datasets) and can be considered the smallest granual of location data. 
* If the desired station is known, all of its data can quickly be viewed.

In [10]:
# read all project stations fetched from the NOAA API
all_project_stations = pd.read_csv('./data/all_project_stations.csv')
all_project_stations.head()

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,2018-10-16,1948-01-01,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,2018-10-16,1996-11-01,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,2018-10-17,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,2018-10-16,1948-01-01,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,2018-09-30,1908-12-01,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table


#### Get the dataypes for wind which we will be downloading the wind data

In [11]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=WIND&limit=56"
response = requests.get(url=url, headers=headers)
noaa_wind_data_types_json = response.json()
noaa_wind_data_types_df = pd.DataFrame(noaa_wind_data_types_json['results'])
wind_data_types = noaa_wind_data_types_df.id

In [12]:
len(wind_data_types)

27

In [13]:
#parameters for weather query
datasetid='GHCND'
datatypeid="AWND"
stationid='GHCND:USW00013969'
units='standard'
startdate='2017-01-01'
enddate='2017-12-31'
limit = 365

In [14]:
def get_wind_data_daily_summary(stationid, datatypeid, units = 'standard', startdate = '2017-01-01', enddate='2017-12-31', limit =365, offset = 25):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=" +\
    datatypeid + "&stationid=" + stationid + "&units=" + units + "&startdate=" + startdate +\
    "&enddate=" + enddate + "&limit=" + str(limit)
    response = requests.get(url=url, headers=headers)
    station_weather_report_json = response.json()
    station_weather_report_df = pd.DataFrame(station_weather_report_json['results'])
    return(station_weather_report_df)

In [15]:
all_project_stations.head()

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,2018-10-16,1948-01-01,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,2018-10-16,1996-11-01,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,2018-10-17,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,2018-10-16,1948-01-01,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,2018-09-30,1908-12-01,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table


In [18]:
remaining_stations = pd.read_csv("./data/remaining_stations.csv")
remaining_stations = remaining_stations[:350]

station_count = 0
query_count = 0
start_date = "2015-01-01"
end_date = "2016-12-31"
for index, row in remaining_stations.iterrows():
    stationid = row['id'].replace("_", ":")
    stationid_dir_name = row["id"]
    projectid = row["p_name"].replace("/","&")
    failed_datatypes = []
    station_count += 1
    for w in wind_data_types:
        try:
            query_count += 1
            print("Project:", projectid, " | Station:", stationid, " | Feature:", w,
                  " | Station Count:", station_count, " | Query Count:", query_count)
            downloaded_data = get_wind_data_daily_summary(stationid, w, units = 'standard',
                                                          startdate=start_date , enddate=end_date, limit=1000)
            file_path = './data/weather/projects/' + projectid + "/" + stationid_dir_name + '/' + w + \
            "_" + start_date + "_" + end_date + ".csv"
            downloaded_data.to_csv(file_path, index=False)
        except:
            failed_datatypes.append(w)
    with open('./data/weather/projects/' + projectid + "/" + stationid_dir_name + '/failed_datatypes' , 'w') as f:
        for item in failed_datatypes:
            f.write("%s\n" % item)

Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: AWND  | Station Count: 1  | Query Count: 1
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: DAWM  | Station Count: 1  | Query Count: 2
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: FMTM  | Station Count: 1  | Query Count: 3
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-1STDIR  | Station Count: 1  | Query Count: 4
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-1STPCT  | Station Count: 1  | Query Count: 5
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-2NDDIR  | Station Count: 1  | Query Count: 6
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-2NDPCT  | Station Count: 1  | Query Count: 7
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-AVGSPD  | Station Count: 1  | Query Count: 8
Project: AG Land 4  | Station: GHCND:USW00014933  | Feature: HLY-WIND-PCTCLM  | Station Count: 1  | Query Count: 

Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: MDWM  | Station Count: 3  | Query Count: 66
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: PGTM  | Station Count: 3  | Query Count: 67
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: TWND  | Station Count: 3  | Query Count: 68
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: WDF1  | Station Count: 3  | Query Count: 69
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: WDF2  | Station Count: 3  | Query Count: 70
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: WDF5  | Station Count: 3  | Query Count: 71
Project: Shakopee Mdewakanton Sioux Community Wind  | Station: GHCND:USW00014922  | Feature: WDFG  | Station Count: 3  | Query Count: 72
Project: Shakopee Mdewakanton Sioux Commu

Project: Wind GEM  | Station: GHCND:USC00133438  | Feature: WSF2  | Station Count: 5  | Query Count: 131
Project: Wind GEM  | Station: GHCND:USC00133438  | Feature: WSF5  | Station Count: 5  | Query Count: 132
Project: Wind GEM  | Station: GHCND:USC00133438  | Feature: WSFG  | Station Count: 5  | Query Count: 133
Project: Wind GEM  | Station: GHCND:USC00133438  | Feature: WSFI  | Station Count: 5  | Query Count: 134
Project: Wind GEM  | Station: GHCND:USC00133438  | Feature: WSFM  | Station Count: 5  | Query Count: 135
Project: Wind GEM  | Station: GHCND:USW00014933  | Feature: AWND  | Station Count: 6  | Query Count: 136
Project: Wind GEM  | Station: GHCND:USW00014933  | Feature: DAWM  | Station Count: 6  | Query Count: 137
Project: Wind GEM  | Station: GHCND:USW00014933  | Feature: FMTM  | Station Count: 6  | Query Count: 138
Project: Wind GEM  | Station: GHCND:USW00014933  | Feature: HLY-WIND-1STDIR  | Station Count: 6  | Query Count: 139
Project: Wind GEM  | Station: GHCND:USW00014

Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WDF5  | Station Count: 8  | Query Count: 206
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WDFG  | Station Count: 8  | Query Count: 207
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WDFI  | Station Count: 8  | Query Count: 208
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WDFM  | Station Count: 8  | Query Count: 209
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WDMV  | Station Count: 8  | Query Count: 210
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WSF1  | Station Count: 8  | Query Count: 211
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WSF2  | Station Count: 8  | Query Count: 212
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WSF5  | Station Count: 8  | Query Count: 213
Project: Wind Walkers  | Station: GHCND:USW00014940  | Feature: WSFG  | Station Count: 8  | Query Count: 214
Project: Wind Walke

Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: HLY-WIND-VCTDIR  | Station Count: 11  | Query Count: 280
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: HLY-WIND-VCTSPD  | Station Count: 11  | Query Count: 281
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: MDWM  | Station Count: 11  | Query Count: 282
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: PGTM  | Station Count: 11  | Query Count: 283
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: TWND  | Station Count: 11  | Query Count: 284
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: WDF1  | Station Count: 11  | Query Count: 285
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: WDF2  | Station Count: 11  | Query Count: 286
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: WDF5  | Station Count: 11  | Query Count: 287
Project: Wiota Wind  | Station: GHCND:USC00133438  | Feature: WDFG  | Station Count: 11  | Query Count: 288
Projec

...

Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WDFM  | Station Count: 94  | Query Count: 2531
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WDMV  | Station Count: 94  | Query Count: 2532
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSF1  | Station Count: 94  | Query Count: 2533
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSF2  | Station Count: 94  | Query Count: 2534
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSF5  | Station Count: 94  | Query Count: 2535
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSFG  | Station Count: 94  | Query Count: 2536
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSFI  | Station Count: 94  | Query Count: 2537
Project: Forward Fontanelle  | Station: GHCND:USW00094991  | Feature: WSFM  | Station Count: 94  | Query Count: 2538
Project: Cumberland Rose  | Station: GHCND:USC00133438  | Featur

Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: HLY-WIND-VCTDIR  | Station Count: 97  | Query Count: 2602
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: HLY-WIND-VCTSPD  | Station Count: 97  | Query Count: 2603
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: MDWM  | Station Count: 97  | Query Count: 2604
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: PGTM  | Station Count: 97  | Query Count: 2605
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: TWND  | Station Count: 97  | Query Count: 2606
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: WDF1  | Station Count: 97  | Query Count: 2607
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: WDF2  | Station Count: 97  | Query Count: 2608
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature: WDF5  | Station Count: 97  | Query Count: 2609
Project: Cumberland Rose  | Station: GHCND:USW00014933  | Feature:

Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: FMTM  | Station Count: 100  | Query Count: 2676
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-1STDIR  | Station Count: 100  | Query Count: 2677
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-1STPCT  | Station Count: 100  | Query Count: 2678
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-2NDDIR  | Station Count: 100  | Query Count: 2679
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-2NDPCT  | Station Count: 100  | Query Count: 2680
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-AVGSPD  | Station Count: 100  | Query Count: 2681
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-PCTCLM  | Station Count: 100  | Query Count: 2682
Project: Crow Lake  | Station: GHCND:USW00014936  | Feature: HLY-WIND-VCTDIR  | Station Count: 100  | Query Count: 2683
Project: Crow Lake  | Station: GHCND:USW00014936  |

Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: MDWM  | Station Count: 102  | Query Count: 2739
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: PGTM  | Station Count: 102  | Query Count: 2740
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: TWND  | Station Count: 102  | Query Count: 2741
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: WDF1  | Station Count: 102  | Query Count: 2742
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: WDF2  | Station Count: 102  | Query Count: 2743
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: WDF5  | Station Count: 102  | Query Count: 2744
Project: City of Soledad Water Reclamation Facility  | Station: GHCND:USW00023259  | Feature: WDFG  | Station Count: 102  | Query Count: 2745
Projec

Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WDFI  | Station Count: 104  | Query Count: 2800
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WDFM  | Station Count: 104  | Query Count: 2801
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WDMV  | Station Count: 104  | Query Count: 2802
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSF1  | Station Count: 104  | Query Count: 2803
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSF2  | Station Count: 104  | Query Count: 2804
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSF5  | Station Count: 104  | Query Count: 2805
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSFG  | Station Count: 104  | Query Count: 2806
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSFI  | Station Count: 104  | Query Count: 2807
Project: Sky Volt  | Station: GHCND:USC00133438  | Feature: WSFM  | Station Count: 104  | Query Count: 2808
Project: Sky Volt  | Station